In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.lang.fr import French
import pandas as pd
import json
import ast
from spacy_langdetect import LanguageDetector
import minibatch
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import numpy as np

In [2]:
nlp = French()

### Datas

In [78]:
def read_jsonl(path_json_file):
    with open(path_json_file, 'r') as json_file:
        json_list = list(json_file)
    return json_list

path_json_file = "/Users/remyadda/Desktop/AD/Projets/ALFRED/Datas/Doccano/formation_json.json1"
DATA = read_jsonl(path_json_file)

In [79]:
DATA[8]

'{"id": 393, "text": "FORMATIONS SAUTDELIGNE  SAUTDELIGNE Preparation en cours formation a distance Dipleme d\'Etat de Comptabilite Gestion DCG - EFC Lyon certifie ISQ OPQF - 2020/2021 SAUTDELIGNE Master 1 SEG mention management parcours Audit Contrele Gestion ACG- ISM Adonai Cotonou certifie ISO 9001. - 2013 SAUTDELIGNE  SAUTDELIGNE Licence AEI parcours Ressources Humaines RH - Universite Paris Est-Creteil. - 2019 SAUTDELIGNE  SAUTDELIGNE Licence professionnelle SEG mention management parcours Audit Contrele Gestion ACG- ISM Adonai Cotonou certifie ISO 9001 - 2012 SAUTDELIGNE Baccalaureat Techniques Quantitatives de Gestion serie G2 Comptabilite Gestion CG - CRPIB Porto-Novo - 2009", "meta": {}, "annotation_approver": "ariebonan", "labels": [[117, 120, "U-Diplome"], [174, 180, "B-Diplome"], [181, 182, "L-Diplome"], [183, 186, "U-DiplomeSpec"], [313, 320, "U-Diplome"], [321, 324, "B-DiplomeSpec"], [421, 428, "B-Diplome"], [429, 444, "L-Diplome"], [445, 448, "U-DiplomeSpec"], [325, 333,

In [82]:
def reformate_datas(TRAIN_DATA):
    """Input: JSON file issu de Doccano
       Output: Datas mis au format du NER --> (text, {entities :})"""
    TRAIN = []
    for element in TRAIN_DATA:
        # null pose pb. On le remplace par un mot qqconque
        element = element.replace('null', '"ok"')
        d = eval(element)
        dictOfLabels = {}
        dictOfLabels['entities'] = d['labels']
        listOfTuples = []
        for element in dictOfLabels['entities']:
            element = tuple(element)
            listOfTuples.append(element)
        dictOfLabels['entities'] =  listOfTuples
        TRAIN.append((d['text'], dictOfLabels))
    return TRAIN

### Reformate + Train Test Split

In [83]:
DATA = reformate_datas(DATA)
print(len(DATA))

250


In [84]:
TRAIN = DATA[:int(0.8*len(DATA))]
TEST = DATA[int(0.8*len(DATA)):]
print('Len train set : ', len(TRAIN))
print('Len test set : ', len(TEST))

Len train set :  200
Len test set :  50


In [85]:
DATA

[('WVU SAUTDELIGNE  SAUTDELIGNE MASTER 2 STRATEGIE COMMERCIALE ET MARKETING RELATIONNEL SAUTDELIGNE MENTION BIEN SAUTDELIGNE  SAUTDELIGNE Sup Career INSEEC - 75010 PARIS SAUTDELIGNE  SAUTDELIGNE 2020 SAUTDELIGNE  SAUTDELIGNE BACHELOR MARKETING ET DEVELOPPEMENT COMMERCIAL SAUTDELIGNE Sup Career INSEEC - 75016 PARIS SAUTDELIGNE 2018 SAUTDELIGNE  SAUTDELIGNE DUT TECHNIQUES DE COMMERCIALISATION SAUTDELIGNE IUT de Sceaux Universite Paris Sud - 92330 SCEAUX SAUTDELIGNE 2017 SAUTDELIGNE  SAUTDELIGNE BACCALAUREAT SCIENTIFIQUE SPE SVT MENTION ASSEZ BIEN SAUTDELIGNE Lycee Leonard de Vinci - 91240 SAINT MICHEL SUR ORGE SAUTDELIGNE 2015',
  {'entities': [(29, 35, 'B-Diplome'),
    (36, 37, 'L-Diplome'),
    (38, 47, 'B-DiplomeSpec'),
    (48, 59, 'I-DiplomeSpec'),
    (60, 62, 'I-DiplomeSpec'),
    (63, 72, 'I-DiplomeSpec'),
    (73, 84, 'L-DiplomeSpec'),
    (222, 230, 'U-Diplome'),
    (231, 240, 'B-DiplomeSpec'),
    (241, 243, 'I-DiplomeSpec'),
    (244, 257, 'I-DiplomeSpec'),
    (258, 268, '

In [86]:
TEST

[('FORMATION SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE 2012-2014 : CIECE Groupe Ecole Superieure de Commerce Exterieur (ESCE) See a AL ee) SAUTDELIGNE Master 2 Strategie et Marketing specialite Commerce International SAUTDELIGNE MBA Specialite Commerce International',
  {'entities': []}),
 ('FORMATION SAUTDELIGNE  SAUTDELIGNE 1996 Baccalaureat Sciences Medico - Sociales SAUTDELIGNE Lycee MARCEL CACHIN, SAINT -- OUEN (93) SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE Informatique : Word, Excel (notions).',
  {'entities': []}),
 ('wae SAUTDELIGNE AUT SAUTDELIGNE muy SAUTDELIGNE ee, SAUTDELIGNE  SAUTDELIGNE Titre Professionnel Gestionnaire Comptable - AFPA Creteil SAUTDELIGNE  SAUTDELIGNE DCG UE: Droit des societes, Management et Contrele de gestion SAUTDELIGNE  SAUTDELIGNE DUT Gestion des Entreprises et Administrations option Fiance - Paris 16 SAUTDELIGNE Baccalaureat Professionnel Comptabilite',
  {'entities': []}),
 ('DIPLOMES SAUTDELIGNE  SAUTDELIGNE Fevrier 2019 SAUTD

### Format Datas apres transformation --> (texte, listes de tuples)

In [39]:
list(set([tuple_[2] for tuple_ in DATA[0][1]['entities']]))

['L-Diplome',
 'U-DiplomeSpec',
 'B-DiplomeSpec',
 'I-DiplomeSpec',
 'B-Diplome',
 'L-DiplomeSpec',
 'U-Diplome']

### Modeling -- Ci dessous 2 differents NER

In [40]:
def train_model_2(LABEL, TRAIN):
    # Setting up the pipeline and entity recognizer.
    nlp = spacy.blank('fr')  # create blank Language class
    print("Created blank 'fr' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner')
    else:
        ner = nlp.get_pipe('ner')
    # Add new entity labels to entity recognizer
    for i in LABEL:
        ner.add_label(i)
    # Inititalizing optimizer
    optimizer = nlp.begin_training()
    # Get names of other pipes to disable them during training to train # only NER and update the weights
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(40):
            random.shuffle(TRAIN)
            losses = {}
            batches = minibatch(TRAIN, 
                                size=compounding(4., 32., 1.001))
            for batch in batches:
                for text, annotations in batch:
                    # create Example
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    # Update the model
                    nlp.update([example], losses=losses, drop=0.3)
    return nlp

In [41]:
LABEL_skills = ["U-SKILLS", "B-SKILLS", "I-SKILLS", "L-SKILLS", "U-ANGLAIS", "U-NIVEAU_ANGLAIS",
    "B-NIVEAU_ANGLAIS", "I-NIVEAU_ANGLAIS", "L-NIVEAU_ANGLAIS", "U-OTHER_LANGUAGE"]
LABEL_exp_pro = ['B-JobTitle','B-DateExp','I-DateExp','L-JobTitle','L-DateExp','I-JobTitle',
 'U-Company','L-Company','B-Company']
LABEL_formation = ['L-Diplome', 'U-DiplomeSpec', 'B-DiplomeSpec', 'I-DiplomeSpec',
 'B-Diplome', 'L-DiplomeSpec', 'U-Diplome']

nlp = train_model_2(LABEL_formation, TRAIN)

Created blank 'fr' model


/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "2012 Chargee de gestion immobiliere SAUTDELIGNE L3..." with entities "[(48, 50, 'U-Diplome'), (5, 12, 'B-DiplomeSpec'), ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Diplomes SAUTDELIGNE BAC-- Marketing SAUTDELIGNE B..." with entities "[(21, 24, 'U-Diplome'), (27, 36, 'U-DiplomeSpec'),...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/remyadda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/sp

### Save a model

In [42]:
def save_model(nlp, output_dir, model_name):
    """Input: nlp (model), directory ou l'on souhaite save le mpdel,
    model_name est le nom qu'on souhaite donner a notre modele"""
    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

save_model(nlp, "/Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3/Modeles", 'ner')

Saved model to /Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3/Modeles


### Test on new datas

In [43]:
def test_ner_model(output_dir, test_text):
    """Permet de tester un NER
    Input: directory du modele et texte a tester ainsi que le texte
    Output: Renvoie les labels predits ainsi que le texte correspondant a chaque label"""
    # Test the saved model
    nlp = spacy.load(output_dir)
    doc = nlp(test_text)
    predicted_labels = []
    corresponding_text = []
    for ent in doc.ents:
        predicted_labels.append(ent.label_)
        corresponding_text.append(ent.text)
    return predicted_labels, corresponding_text

In [44]:
TEST[0]

('FORMATION SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE 2012-2014 : CIECE Groupe Ecole Superieure de Commerce Exterieur (ESCE) See a AL ee) SAUTDELIGNE Master 2 Strategie et Marketing specialite Commerce International SAUTDELIGNE MBA Specialite Commerce International',
 {'entities': []})

In [60]:
def retrieve_trueLabelAndWord(DATASET):
    """Permet de recuperer deux listes: True label, et le word correspondant
    Ceci permettra de comparer les valeurs reelles aux valeurs predites du NER"""
    true_labels = []
    corresponding_text = []
    for element in DATASET:
        print(element)
        # on retrieve le texte
        text = element[0]
        label = element[1]
        label = label['entities']
        for e in label:
            start = e[0]
            end = e[1]
            target = e[2]
            true_labels.append(target)
            corresponding_word = text[start:end]
            corresponding_text.append(corresponding_word)
    return true_labels, corresponding_text

In [61]:
def X_test(DATASET):
    """Permet de recuperer le texte afin d'effectuer les predictions"""
    texts = []
    for element in DATASET:
        text = element[0]
        texts.append(text)
    return texts

In [62]:
Xtest = X_test(TEST)
print(Xtest[0])

FORMATION SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE 2012-2014 : CIECE Groupe Ecole Superieure de Commerce Exterieur (ESCE) See a AL ee) SAUTDELIGNE Master 2 Strategie et Marketing specialite Commerce International SAUTDELIGNE MBA Specialite Commerce International


#### 1- Récupération des tokens et labels que le modèle devrait être capable de prédire (True tuples on les appelera)

In [63]:
true_labels, true_corresponding_words = retrieve_trueLabelAndWord(TEST)
print('True labels : ', true_labels[0], ' --> True corresponding word : ',true_corresponding_words[0])
# Create list of tuples (word, label_predicted) --> useful for metrics

def make_tuples(true_token, true_labels):
    """Input: la liste de mots (token) ainsi que la liste de labels correspondante
       Output: liste de tuples ou chaque tuple est de la forme (token, label)"""
    true_tuple = []
    for w, p in zip(true_token, true_labels):
        true_tuple.append((w, p))

    print(len(true_tuple), 'tuples')
    return true_tuple

# Nbe de tuples a trouver
true_tuple = make_tuples(true_corresponding_words, true_labels)

('FORMATION SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE 2012-2014 : CIECE Groupe Ecole Superieure de Commerce Exterieur (ESCE) See a AL ee) SAUTDELIGNE Master 2 Strategie et Marketing specialite Commerce International SAUTDELIGNE MBA Specialite Commerce International', {'entities': []})
('FORMATION SAUTDELIGNE  SAUTDELIGNE 1996 Baccalaureat Sciences Medico - Sociales SAUTDELIGNE Lycee MARCEL CACHIN, SAINT -- OUEN (93) SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE Informatique : Word, Excel (notions).', {'entities': []})
('wae SAUTDELIGNE AUT SAUTDELIGNE muy SAUTDELIGNE ee, SAUTDELIGNE  SAUTDELIGNE Titre Professionnel Gestionnaire Comptable - AFPA Creteil SAUTDELIGNE  SAUTDELIGNE DCG UE: Droit des societes, Management et Contrele de gestion SAUTDELIGNE  SAUTDELIGNE DUT Gestion des Entreprises et Administrations option Fiance - Paris 16 SAUTDELIGNE Baccalaureat Professionnel Comptabilite', {'entities': []})
('DIPLOMES SAUTDELIGNE  SAUTDELIGNE Fevrier 2019 SAUTDELIGNE  SAUTD

### Predictions

In [49]:
# On predit chaque texte de Xtest
output_dir = "/Users/remyadda/Desktop/AD/Projets/ALFRED/ALFRED_PROJECT/Model_NER_3/Modeles"
preds = []
predicted_corresponding_word_list = []
for text in Xtest:
    predictions, predicted_corresponding_word = test_ner_model(output_dir, text)
    preds.extend(predictions)
    predicted_corresponding_word_list.extend(predicted_corresponding_word)

#### 2 - Récupération des tokens et labels que le modèle a prédit (predicted tuples on les appelera)

In [50]:
# Create list of tuples (word, label_predicted) --> useful for metrics
# Nbe de tuples prédits
predicted_tuple = make_tuples(predicted_corresponding_word_list, preds)

656 tuples


### Defintion des Metrics

In [51]:
# Explication : supposons que dans le Test set il y ait 200 mots a labeliser, et que le modele en trouve 100, 
# alors, le coverage serait de 50%. Certes toutes les predictions peuvent etre bonnes, mais il en manque la moitié.
def get_metrics(y_pred, y_true):
    """Input: y_pred et y_true sous forme de listes de tuple (texte, label)
    Renvoie le poucentage de bons tuples (texte, label) détectés par le modele"""
    y_true_copy = y_true.copy()
    y_pred_copy = y_pred.copy()
    wrong_tuples = []
    good_tuples = []
    count = 0
    for tuple_ in y_pred_copy:
        count += 1
        if tuple_ in y_true_copy:
            y_true_copy.remove(tuple_)
            good_tuples.append(tuple_)
        else:
            # tuple qui na pas lieu d'exister
            wrong_tuples.append(tuple_)
    #print(count)
    accuracy_1 = np.round(len(good_tuples)/len(y_pred), 3) # cb de bonnes predictions par rap a ce que je predis
    accuracy_2 = np.round(len(good_tuples)/len(y_true), 3) # % cb de bonnes predictions par rap a ce quil fallait trouver
    
    #lets take a look on bad predictions
    # erreur de type 1 : mot trouvé, mais prédiction fausse
    label_error = {} # on spotera les labels où le modèle fait erreur
    count_error_I = 0
    for tuple_ in wrong_tuples:
        for TrueTuple in y_true_copy:
            if tuple_[0] == TrueTuple[0]:
                count_error_I += 1
                if (tuple_[1], TrueTuple[1]) not in label_error:
                    label_error[(tuple_[1], TrueTuple[1])] = 1
                else:
                    label_error[(tuple_[1], TrueTuple[1])] += 1
                y_true_copy.remove(TrueTuple)
                break
        # Si c'est pas une erreur de type I c'est qu'il s'agit d'une erreur de type II 
        # cad un mot releve qui n'aurait jamais du l'etre
    count_error_II = len(wrong_tuples) - count_error_I
    coverage = np.round(len(y_true) / len(y_pred) * 100, 1)
    coverage = np.round(np.abs(100 - coverage), 1)
    print("Nombre de mots à prédire pour l'ensemble du test set : ", len(y_true))
    print("Nombre de mots prédits par le NER pour l'ensemble du test set : ", len(y_pred))
    print('Coverage : ', coverage, '% de difference')
    print('----------------------------------------------------------------------')
    print(len(good_tuples), ' mots ont bien été relevés ET prédits par le NER')
    print(len(wrong_tuples), " mots ont mal été prédits (erreur de type I ou type II)")
    print('Bonnes prédictions + Mauvaises prédictions =', len(good_tuples)+len(wrong_tuples))
    print('----------------------------------------------------------------------')
    print('Accuracy I (bonne pred / total pred) : ', accuracy_1*100, ' %')
    print('Accuracy II (bonne pred / total expected pred) : ', accuracy_2*100, ' %')
    print('----------------------------------------------------------------------')
    print('Nbe erreur type I : ', count_error_I, ' (mots bien relevés mais mal prédits)')
    print('Nbe erreur type II : ', count_error_II, " (mots qui n'auraient pas du être prédits)")
    print('----------------------------------------------------------------------')
    print("Confusions faites par l'algo (prediction, true_label): \n", label_error)

#### Lets check our results

In [52]:
get_metrics(predicted_tuple, true_tuple)

Nombre de mots à prédire pour l'ensemble du test set :  321
Nombre de mots prédits par le NER pour l'ensemble du test set :  656
Coverage :  51.1 % de difference
----------------------------------------------------------------------
223  mots ont bien été relevés ET prédits par le NER
433  mots ont mal été prédits (erreur de type I ou type II)
Bonnes prédictions + Mauvaises prédictions = 656
----------------------------------------------------------------------
Accuracy I (bonne pred / total pred) :  34.0  %
Accuracy II (bonne pred / total expected pred) :  69.5  %
----------------------------------------------------------------------
Nbe erreur type I :  30  (mots bien relevés mais mal prédits)
Nbe erreur type II :  403  (mots qui n'auraient pas du être prédits)
----------------------------------------------------------------------
Confusions faites par l'algo (prediction, true_label): 
 {('L-DiplomeSpec', 'L-Diplome'): 2, ('I-DiplomeSpec', 'I-Diplome'): 4, ('U-Diplome', 'B-Diplome'):

In [53]:
true_tuple

[('M.B.A', 'U-Diplome'),
 ('Management', 'B-DiplomeSpec'),
 ('International', 'I-DiplomeSpec'),
 ('Business', 'L-DiplomeSpec'),
 ('Master', 'U-Diplome'),
 ('Manager', 'B-DiplomeSpec'),
 ('du', 'I-Diplome'),
 ('Developpement', 'L-DiplomeSpec'),
 ('Commercial', 'L-DiplomeSpec'),
 ('Master', 'U-Diplome'),
 ('Management', 'B-DiplomeSpec'),
 ("d'entreprise", 'L-DiplomeSpec'),
 ('centre', 'U-DiplomeSpec'),
 ('profit', 'L-DiplomeSpec'),
 ('Bachelor', 'U-Diplome'),
 ('Responsable', 'B-DiplomeSpec'),
 ('Projet', 'I-DiplomeSpec'),
 ('et', 'I-DiplomeSpec'),
 ('Developpement', 'I-DiplomeSpec'),
 ("d'affaires", 'L-DiplomeSpec'),
 ('B.T.S', 'U-Diplome'),
 ('Management', 'U-DiplomeSpec'),
 ('Unites', 'I-DiplomeSpec'),
 ('commerciales', 'L-DiplomeSpec'),
 ('des', 'I-DiplomeSpec'),
 ('Licence', 'B-Diplome'),
 ('professionnelle', 'L-Diplome'),
 ('Management', 'B-DiplomeSpec'),
 ('Organisations', 'I-DiplomeSpec'),
 ('des', 'I-DiplomeSpec'),
 ('en', 'I-DiplomeSpec'),
 ('Reseau', 'L-DiplomeSpec'),
 ('Breve

In [55]:
Xtest

['FORMATION SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE 2012-2014 : CIECE Groupe Ecole Superieure de Commerce Exterieur (ESCE) See a AL ee) SAUTDELIGNE Master 2 Strategie et Marketing specialite Commerce International SAUTDELIGNE MBA Specialite Commerce International',
 'FORMATION SAUTDELIGNE  SAUTDELIGNE 1996 Baccalaureat Sciences Medico - Sociales SAUTDELIGNE Lycee MARCEL CACHIN, SAINT -- OUEN (93) SAUTDELIGNE  SAUTDELIGNE   SAUTDELIGNE  SAUTDELIGNE Informatique : Word, Excel (notions).',
 'wae SAUTDELIGNE AUT SAUTDELIGNE muy SAUTDELIGNE ee, SAUTDELIGNE  SAUTDELIGNE Titre Professionnel Gestionnaire Comptable - AFPA Creteil SAUTDELIGNE  SAUTDELIGNE DCG UE: Droit des societes, Management et Contrele de gestion SAUTDELIGNE  SAUTDELIGNE DUT Gestion des Entreprises et Administrations option Fiance - Paris 16 SAUTDELIGNE Baccalaureat Professionnel Comptabilite',
 'DIPLOMES SAUTDELIGNE  SAUTDELIGNE Fevrier 2019 SAUTDELIGNE  SAUTDELIGNE BTS ASSISTANT DE MANAGER SAUTDELIGNE Dava, Pari

In [54]:
predicted_tuple

[('Master', 'B-Diplome'),
 ('2', 'L-Diplome'),
 ('Strategie', 'B-DiplomeSpec'),
 ('et', 'I-DiplomeSpec'),
 ('Commerce', 'I-DiplomeSpec'),
 ('International', 'L-DiplomeSpec'),
 ('MBA', 'U-Diplome'),
 ('Commerce', 'B-DiplomeSpec'),
 ('International', 'L-DiplomeSpec'),
 ('Baccalaureat', 'U-Diplome'),
 ('Sciences', 'B-DiplomeSpec'),
 ('Medico', 'I-DiplomeSpec'),
 ('-', 'I-DiplomeSpec'),
 ('Sociales', 'L-DiplomeSpec'),
 ('Titre', 'B-Diplome'),
 ('Professionnel', 'L-Diplome'),
 ('Gestionnaire', 'B-DiplomeSpec'),
 ('Comptable', 'L-DiplomeSpec'),
 ('DCG', 'U-Diplome'),
 ('Droit', 'B-DiplomeSpec'),
 ('des', 'I-DiplomeSpec'),
 ('societes', 'I-DiplomeSpec'),
 ('DUT', 'U-Diplome'),
 ('Gestion', 'B-DiplomeSpec'),
 ('des', 'I-DiplomeSpec'),
 ('Entreprises', 'I-DiplomeSpec'),
 ('et', 'I-DiplomeSpec'),
 ('Administrations', 'L-DiplomeSpec'),
 ('Baccalaureat', 'B-Diplome'),
 ('Professionnel', 'L-Diplome'),
 ('Comptabilite', 'U-DiplomeSpec'),
 ('BTS', 'U-Diplome'),
 ('ASSISTANT', 'B-DiplomeSpec'),
 ('DE'

#### Pour appliquer cette fonction metric, il faut d'abord diviser votre DATASET en TRAIN et TEST.
    Puis, dans le cas ou votre NER prend ceci en INPUT : 

    INPUT :
    ('LOGICIELS MAITRISES: WORD -- EXCEL -- SAGE -- POWERPOINT -- OUTLOOK',
     {'entities': [(21, 25, 'U-SKILLS'),
       (29, 34, 'U-SKILLS'),
       (38, 42, 'U-SKILLS'),
       (46, 56, 'U-SKILLS'),
       (60, 67, 'U-SKILLS')]}), 
   
    entrainez votre NER sur le train set.
    
    Afin de tester ses performances sur le test, recuperer les mots ainsi que les preditions qu'il aurait du       pouvoir effectuer dans le meilleur des cas, pour cela, utilisez la fonction "retrieve_trueLabelAndWord" qui prend en input l'INPUT ci dessus, et renvoie deux listes : true label et mots à predire.
    
        Utiliser ensuite la fonction "make_tuple" afin de merge ces deux listes en une seule liste de tuples.
        On utilise deux fois cette fonction : 
        - pour les reels labels a trouver
        - pour les prediction effectuées par l'algo
        
        Enfin on applique la fonction get_metrics
    